In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tokenizer
!pip install transformers
!pip install pyunpack
!pip install patool
from pyunpack import Archive
from zipfile import ZipFile
import shutil
#as an aware datetime
from datetime import datetime, timezone
#or from pytz database
import pytz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 6.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 15.1 MB/s 
     |████████████████████████████████| 7.6 MB 66.4 MB/s 
     |████████████████████████████████| 163 kB 51.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 77 kB 2.0 MB/s 


In [3]:
import tensorflow as tf
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer,GPT2LMHeadModel
# loading tokenizer from the saved model path
tokenizer = GPT2Tokenizer.from_pretrained('NlpHUST/gpt2-vietnamese')
# tokenizer = GPT2Tokenizer.from_pretrained('./tokenized_data/')
model = TFGPT2LMHeadModel.from_pretrained('NlpHUST/gpt2-vietnamese',from_pt=True)

Downloading:   0%|          | 0.00/854k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/512k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/215 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/884 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/510M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['h.6.attn.masked_bias', 'h.8.attn.masked_bias', 'h.3.attn.masked_bias', 'h.1.attn.masked_bias', 'h.0.attn.masked_bias', 'h.7.attn.masked_bias', 'h.4.attn.masked_bias', 'h.9.attn.masked_bias', 'h.2.attn.masked_bias', 'h.11.attn.masked_bias', 'h.5.attn.masked_bias', 'h.10.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was tra

In [4]:
!mkdir dataset
!cp "/content/drive/MyDrive/Transformer-text generation-Thien-Trung-Kiet/dataset.txt" "./dataset"

In [5]:
from pathlib import Path
paths = [str(x) for x in Path("./dataset").glob("**/*.txt")]

IGNORE

In [ ]:
# single_string = ''
# for filename in paths:
#   with open(filename, "r", encoding='utf-8') as f:
#    x = f.read()
#   single_string += x + tokenizer.eos_token

In [ ]:
# string_tokenized = tokenizer.encode(single_string)

In [ ]:
# import json
# with open("string_tokenized.txt", "w") as f:
#   f.write(json.dumps(string_tokenized))
#   f.close()

In [ ]:
# !cp string_tokenized.txt "/content/drive/MyDrive/Transformer-text generation-Thien-Trung-Kiet/"

DO NOT IGNORE

In [6]:
import json
string_tokenized = ''
# get the tokenString from drive
!cp '/content/drive/MyDrive/Transformer-text generation-Thien-Trung-Kiet/string_tokenized.txt' string_tokenized.txt
with open('string_tokenized.txt', 'r') as f:
    a = json.loads(f.read())
    string_tokenized = a

In [7]:
examples = []
block_size = 100
# 16-24
BATCH_SIZE = 16

BUFFER_SIZE = len(paths)
for i in range(0, len(string_tokenized) - block_size + 1, block_size):
  examples.append(string_tokenized[i:i + block_size])
inputs, labels = [], []
for ex in examples:
  inputs.append(ex[:-1])
  labels.append(ex[1:])

In [9]:
dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [10]:
full_ds_size = len(dataset)
train_ds_size = int(0.80 * full_ds_size)
valid_ds_size = int(0.10 * full_ds_size)

train_ds = dataset.take(train_ds_size)
remaining = dataset.skip(train_ds_size)  
valid_ds = remaining.take(valid_ds_size)
test_ds = remaining.skip(valid_ds_size)

In [11]:
# defining our optimizer
# learning rate should do 3e-5 or default value 0.001
# or should use (0.001, 0.9, 0.999, 10^-8) for (learning_rate, beta1, beta2, epsilon).
# Learning rates 0.0005, 0.001, 0.00146 performed best or 10^-3 to 10^-5
# optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, epsilon=1e-8, clipnorm=1.0)

# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])

DOWNLOAD CHECKPOINT IF ALREADY HAVE (REMEMBER TO SHARE THE CHECKPOINT IN DRIVE)

---



In [12]:
import os
checkpoint_path = "training/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [13]:
!mkdir training
# download checkpoint
!gdown --id 1-08NPREyn_ENeuiQKOLPm5h3CxH_tk71
!unzip './12-11|11-59.zip' -d './training'

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-08NPREyn_ENeuiQKOLPm5h3CxH_tk71
To: /content/12-11|11-59.zip
100% 1.38G/1.38G [00:15<00:00, 89.3MB/s]
Archive:  ./12-11|11-59.zip
  inflating: ./training/cp.ckpt.index  
  inflating: ./training/checkpoint   
  inflating: ./training/cp.ckpt.data-00000-of-00001  


In [14]:
# download history
try:
  shutil.copy('/content/drive/MyDrive/Transformer-text generation-Thien-Trung-Kiet/history1.npy', "./")
except:
  print('cant download')

In [15]:
# Loads the weights OF THE CHECKPOINT 
model.load_weights(checkpoint_path)

In [16]:
import tensorflow as tf
from tensorflow import keras
import shutil
#as an aware datetime
from datetime import datetime, timezone
import numpy as np
import matplotlib.pyplot as plt

#or from pytz database
import pytz
class CustomCallback(keras.callbacks.Callback):
  def __init__(self, patience=0):
    # download history
    super(CustomCallback, self).__init__()
    try:
      history1=np.load('history1.npy',allow_pickle='TRUE').item()
      self.history = {
        'loss': history1['loss'],
        'accuracy': history1['accuracy'],
        'val_loss': history1['val_loss'],
        'val_accuracy': history1['val_accuracy']
      }
    except : 
      self.history = {'loss': [],'accuracy': [],'val_loss': [],'val_accuracy': []}

  def on_epoch_end(self, epoch, logs={}):
    self.history['loss'].append(logs.get('loss'))
    self.history['accuracy'].append(logs.get('accuracy'))
    self.history['val_loss'].append(logs.get('val_loss'))
    self.history['val_accuracy'].append(logs.get('val_accuracy'))
    np.save('history1.npy',self.history)
    try:
      os.remove('/content/drive/MyDrive/Transformer-text generation-Thien-Trung-Kiet/history1.npy')
    except:
      print("doesn't have history")
    shutil.copy('./history1.npy', "/content/drive/MyDrive/Transformer-text generation-Thien-Trung-Kiet/")
    tz = pytz.timezone('Asia/Bangkok')
    Asia_now = datetime.now(tz).strftime("%d-%m|%H-%M")
    print("saving to drive!")
    shutil.make_archive(Asia_now, 'zip', './training/')
    location = './' + Asia_now + ".zip"
    print("copy file to driver: "+location)
    shutil.copy(location, "/content/drive/MyDrive/Transformer-text generation-Thien-Trung-Kiet/training-checkpoint/")

In [17]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
#create callback on save
callbacks = [cp_callback, CustomCallback()]

In [ ]:
# Train the model with the new callback
# number of epoch need to be run.
num_epoch = 35
history = model.fit(train_ds,validation_data=valid_ds, epochs=num_epoch,shuffle=True,callbacks=callbacks)  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.


Epoch 1/35
3465/3465 [==============================] - ETA: 0s - loss: 2.9335 - accuracy: 0.4177
Epoch 1: saving model to training/cp.ckpt
saving to drive!
copy file to driver: ./12-11|13-12.zip
3465/3465 [==============================] - 2043s 583ms/step - loss: 2.9335 - accuracy: 0.4177 - val_loss: 3.2727 - val_accuracy: 0.3793
Epoch 2/35
3465/3465 [==============================] - ETA: 0s - loss: 2.5927 - accuracy: 0.4658
Epoch 2: saving model to training/cp.ckpt
saving to drive!
copy file to driver: ./12-11|13-47.zip
3465/3465 [==============================] - 2028s 585ms/step - loss: 2.5927 - accuracy: 0.4658 - val_loss: 3.4064 - val_accuracy: 0.3727
Epoch 3/35
3465/3465 [==============================] - ETA: 0s - loss: 2.3264 - accuracy: 0.5059
Epoch 3: saving model to training/cp.ckpt
saving to drive!
copy file to driver: ./12-11|14-21.zip
3465/3465 [==============================] - 2033s 587ms/step - loss: 2.3264 - accuracy: 0.5059 - val_loss: 3.5771 - val_accuracy: 0.365

DISCONNECT

In [ ]:
from google.colab import runtime
runtime.unassign()

TEST MODEL

In [ ]:
text = "Chủ quán chỗ này là bao nhiêu tiền?"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# input_ids = tokenizer.encode(text)


In [ ]:
output = model.generate(
    input_ids,
    pad_token_id=50256,
    num_beams=5, 
    early_stopping=True,
    max_length=100,
    no_repeat_ngram_size=2,
    num_return_sequences=1)

In [ ]:
for i, sample_output in enumerate(output):
    print(">> Generated text {}\n\n{}".format(i+1, tokenizer.decode(output[i],skip_special_tokens=True)))
    print('\n---')

>> Generated text 1

Chủ quán chỗ này là bao nhiêu tiền? Vi Tiểu Bảo đáp: - Cái đó...
cái đó không phải là tiền.
Gã nói tới đây, đột nhiên một tiếng "bốp" vang lên, một thanh trường kiếm từ trong tay gã bắn ra như một mũi phi chùy, nhằm đâm vào vai bên trái của Phương Di và làm cho nàng phải đau đớn không chịu nổi, bật tiếng la hoảng: – Trời ơi!...
Vi Tiểu bảo giật mình kinh hãi nghĩ thầm : -

---


In [ ]:
text = "Ta nên làm gì đây"
# encoding the input text
input_ids = tokenizer.encode(text, return_tensors='tf')
# input_ids = tokenizer.encode(text)
output = model.generate(
    input_ids,
    pad_token_id=50256,
    num_beams=5, 
    early_stopping=True,
    max_length=100,
    no_repeat_ngram_size=2,
    num_return_sequences=1)

In [ ]:
for i, sample_output in enumerate(output):
    print(">> Generated text {}\n\n{}".format(i+1, tokenizer.decode(output[i],skip_special_tokens=True)))
    print('\n---')

>> Generated text 1

Ta nên làm gì đây? Vi Tiểu Bảo đáp: - Tại hạ không biết làm thế nào được.
Vi hương chủ có thể nói cho tại hạ nghe một câu được chăng? Gã nói rồi đứng yên không nói gì nữa, nhưng trong lòng rất lấy làm kỳ, không hiểu tại sao gã lại nói câu này với một vị cao tăng chùa Thiếu Lâm như vậy? Nhưng gã nghĩ tới đây không khỏi giật mình kinh hãi, bụng bảo dạ : - Té ra sư phụ đã nói vậy.

---
